In [ ]:
try:
    import requests
    import tweepy
except:
    !pip install requests
    !pip install tweepy

In [3]:
#from __future__ import print_function
#import logging
import errno
import sys
import os
import time
import requests
import json
from urlparse import parse_qs
try:
    #python2
    from urllib import urlencode
except ImportError:
    #python3
    from urllib.parse import urlencode
try:
    #python2
    from urlparse import urlparse, urlunparse
except ImportError:
    #python3
    from urllib.parse import urlparse, urlunparse
from pprint import pprint

#from misc import helper
#from misc.config import config
import tweepy
import codecs
#reload(sys)
#sys.setdefaultencoding('utf8')


# Display progress logs on stdout
#logging.basicConfig(level=logging.INFO,
#                    format='%(asctime)s %(levelname)s %(message)s')

ModuleNotFoundError: No module named 'urlparse'

# 1. Check authentication and build API object for calls

In [ ]:
CONSUMER_KEY = "bobVxy9mbwl3C1gnoQ61higKj"
CONSUMER_SECRET = "6S57qgv3xiM3kuS7WIfeQTODbqKc6fHjWxpL9xLyvFwjLTMjEK"
auth = tweepy.AppAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

if not api:
    print("no authentication")
    sys.exit(1)
else:
    print("API has been set up")

# 2. Perform API pulling

## 2.1 Generate the different requests for the topics and emoticons
Result will be a request query for every combination of topic with emoticon(positive/negative)

In [ ]:
topics = {
    "Series":[
        "Breaking Bad",
        "Stranger Things",
        "Two and a half men",
        "The big bang theory",
        "Game of Thrones",
        "Better Call Saul",
        "Sherlock",
        "True Detective",
        "The Walking Dead",
        "House of Cards",
        "How I met your Mother",
        "Black Mirror",
        "Lost in Space",
        "Riverdale",
        "House of Cards",
        "Westworld",
        "Suits",
        "Supernatural",
        "Prison Break",
        "Homeland",
        "Dr. House",
        "Law and Order",
        "TV Show",
        "Netflix"
    ],
    "Movies":[
        "Avengers",
        "Marvel",
        "Fifty Shades of Grey",
        "A Quiet Place",
        "Anihilation",
        "Lord of the Rings",
        "Star Wars",
        "Infinity War",
        "Spider Man",
        "Guardians of the Galaxy",
        "Black Panther",
        "Iron Man",
        "Deadpool",
        "Red Sparrow",
        "Batman"
    ],
    "Technology":[
        "iPhone",
        "iPad",
        "Android",
        "Apple",
        "Samsung",
        "Huawei",
        "Alexa",
        "Siri",
        "GoogleHome",
        "Dell",
        "HP",
        "Siemens",
        "Google",
        "Microsoft",
        "Playstation"
    ],
    "FastFood":[
        "McDonalds",
        "Burger King",
        "Pizza Hut",
        "Dunkin Donuts",
        "KFC",
        "Starbucks",
        "Subway",
        "Taco Bell",
        "Wendys",
        "Dominos",
        "Burger",
        "Pizza",
        "Pasta",
        "Foodporn",
        "bbq",
        "barbecue",
        "breakfast",
        "dinner",
        "lunch",
        "brunch",
        "Food"
    ],
    "Musician":[
        "Drake",
        "Dua Lipa",
        "TheWeeknd",
        "Kendrick Lamar",
        "Nicki Minaj",
    #    #"Avicii",
        "Justin Bieber",
        "Lady Gaga",
        "Katy Perry",
        "Rihanna",
        "Taylor Swift",
        "Bruno Mars",
        "Adele",
        "Wiz Khalifa",
        "Justin Timberlake",
        "Rock",
        "Pop",
        "Hiphop",
        "Rap",
        "Jazz",
        "Techno",
        "Coachella",
        "Reggae",
        "Blues",
        "Punk"
    ],
    "USPolitican":[
        "Trump",
        "Obama",
        "Schwarzenegger",
        "Romney",
        "Palin",
        "Clinton",
        "Biden",
        "McCain",
        "Bernie Sanders",
        "Brian Schatz",
        "Michael Enzi",
        "Politic",
        "Senator",
        "President"
    ],
    "CarManufacturer":[
        "Chrysler",
        "Dodge",
        "Jeep",
        "Ford",
        "Cadillac",
        "Chevrolet",
        "Tesla",
        "Ferrari",
        "Land Rover",
        "Porsche",
        "Volkswagen",
        "Nissan",
        "Audi",
        "BMW",
        "Toyota",
        "Car",
        "V8",
        "Mustang",
        "Vehicle"
    ],
    "Soccer":[
        "Ronaldo",
        "Champions League",
        "Neymar",
        "Özil",
        "Andrés Iniesta",
        "Gerard Piqué",
        "Rooney",
        "Suarez",
        "Radamel Falcao",
        "Gareth Bale",
        "James Rodríguez",
        "FC Barcelona",
        "Real Madrid",
        "Arsenal",
        "Chelsea",
        "Bundesliga",
        "nba",
        "nfl",
        "basketball",
        "football",
        "soccer",
        "tennis",
        "golf",
        "rugby",
        "icehockey",
        "paintball",
        "hockey",
        "Roger Federer",
        "Tom Brady"
    ]
}
emoticons = {
    "positive":":)",
    "negative":":("
}

twitter_requests = {}
for topic in topics:
    seed_topics = topics[topic]
    for emoticon_text,emoticon_symbol in emoticons.iteritems():
        helper_request_key = topic+'_'+emoticon_text
        """
        #################################################################################
        Build the search query for the current combination of topic and negative/positive
        #################################################################################
        """
        temp_queries = []
        for seed_topic in seed_topics:
            search_query_parts = []
            #just add double quotes
            search_query_parts.append('"'+seed_topic+'"')
            #build list of emoticons based on current positive or negative
            search_query_parts.append(emoticon_symbol)
            #combine all by joining the parts
            search_query = ' '.join(search_query_parts)
            #add it to the list of requests which should be performed
            temp_queries.append(search_query)
        twitter_requests[helper_request_key] = temp_queries

## 2.2 Perform the real API pulling based on the previously generated requests

In [1]:
default_parameters = {
    'lang':'en',
    'result_type':'recent',
    'tweet_mode':'extended', #due to recent twitter changes, tweets with more than 140 characters are allowed
    'include_entities':True, #used for replacing content with placeholders
    'count':100 # this is the max the API permits
}
data_dir = 'data/raw'
tweet_storage = {}
for request_key,request_seedtopics in twitter_requests.items():
    #split they key ('topic_negative') into both parts ['topic','negative'] for further processing and file storage
    request_key_parts = request_key.split('_')
    for seedtopic in request_seedtopics:
        #create the filename for the pulled tweets of this execution. name is unique due to current timestamp. => <time>.<ext>
        #time stamp will be used for identifying the most recent pulled tweets in subsequent executions
        current_file_name = str(int(time.time()))+"_"+seedtopic+".json"
        #joins the complete path => data/<topic>/<positive/negative>/<time>.<ext>
        current_file_path = os.path.join(data_dir,request_key_parts[0],request_key_parts[1],current_file_name)
        #create the directory if not exists. this is placed at the beginning of the pulling because if an error would occur, it is better to have it at the beginning than at the end of the execution
        path = os.path.dirname(current_file_path)
        try:
            os.makedirs(path)
        except OSError as e:
            if e.errno == errno.EEXIST and os.path.isdir(path):
                pass
            else:
                raise
        
        """
        #######################################################
        2.2.1 Perform requests for the different search queries
        #######################################################
        """
        maxTweets = 1000000 #maximum number of tweets for a single script execution, its high to guarantee to receive all tweets within the last 7 days
    
        tweet_storage = {}
        print
        print('#'*46)
        print("Downloading max {0} tweets for {1} | {2}".format(maxTweets,request_key,seedtopic))
        print('#'*46)
        for tweet in tweepy.Cursor(api.search,q=seedtopic,**default_parameters).items(maxTweets):
            #add tweet to storage
            tweet_storage[tweet.id_str] = tweet._json
            current_length = len(tweet_storage)
            if current_length%100 == 0:
                print("Downloaded {0} tweets so far".format(current_length))
        print("Downloaded {0} tweets".format(current_length)) 
        #save tweets to file
        with open(current_file_path,'wb') as f:
            json.dump(tweet_storage, codecs.getwriter('utf-8')(f), ensure_ascii=False)
    
        #free memory + empty for next topic
        tweet_storage = {}

NameError: name 'twitter_requests' is not defined